In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from splinter import Browser
from time import sleep

In [2]:
def init_browser():
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

# NASA Mars News

In [3]:
nasa_mars_url = "https://mars.nasa.gov/news/"

browser = init_browser()
browser.visit(nasa_mars_url)
sleep(5)
soup = BeautifulSoup(browser.html, 'html.parser')

In [4]:
news_title = soup.find_all("div", class_="content_title")[0].get_text()
news_title

'Mars Now'

In [5]:
news_paragraph = soup.find_all("div", class_="article_teaser_body")[0].get_text()
news_paragraph

'NASA chose a seventh-grader from Virginia as winner of the agency\'s "Name the Rover" essay contest. Alexander Mather\'s entry for "Perseverance" was voted tops among 28,000 entries. '

In [6]:
browser.quit()

# JPL Mars Space Images - Featured Image

In [7]:
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

browser = init_browser()
browser.visit(jpl_url)

In [8]:
browser.click_link_by_partial_text("FULL IMAGE")
# add in lag to allow loading
sleep(5)
browser.click_link_by_partial_text("more info")

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/splinter/driver/webdriver/__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [9]:
browser.click_link_by_partial_text(".jpg")
featured_image_url = browser.url
featured_image_url

'https://photojournal.jpl.nasa.gov/jpeg/PIA07137.jpg'

In [10]:
browser.quit()

# Mars Weather

In [20]:
twitter_url = "https://twitter.com/marswxreport?lang=en"

# use requests library to read in Twitter page
twitter_page = requests.get(twitter_url)
soup = BeautifulSoup(twitter_page.content, 'html.parser')

In [25]:
# find first tweet and clean it
tweet = soup.find("div", class_='stream')
#news_title = tweet.find("div", class_="content_title").text
mars_weather = tweet.find("div", class_ ="js-tweet-text-container").text
print(mars_weather)


Happy #StPatricksDay!

To celebrate, here's an image of Olympus Mons on #Mars—a volcano so big that, if you drove the equivalent distance from Malin Head to Mizen Head in Ireland, you'd *still* be on that damned volcano.pic.twitter.com/4BB13WBBiq



# Mars Facts

In [13]:
mars_facts_url = "https://space-facts.com/mars/"

# use pandas to read table directly from URL
mars_facts_df = pd.read_html(mars_facts_url)[1]

In [14]:
# clean up resulting DataFrame
mars_facts_df.columns = ["description", "value"]
mars_facts_df["description"] = mars_facts_df["description"].str.replace(":","")
mars_facts_df

,description,value
0,Equatorial Diameter,"6,792 km"
1,Polar Diameter,"6,752 km"
2,Mass,6.39 × 10^23 kg (0.11 Earths)
3,Moons,2 (Phobos & Deimos)
4,Orbit Distance,"227,943,824 km (1.38 AU)"
5,Orbit Period,687 days (1.9 years)
6,Surface Temperature,-87 to -5 °C
7,First Record,2nd millennium BC
8,Recorded By,Egyptian astronomers


In [15]:
# convert to HTML string
mars_facts_df.set_index("description", inplace=True)
mars_facts_html = mars_facts_df.to_html()
print(mars_facts_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>value</th>
    </tr>
    <tr>
      <th>description</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


# Mars Hemispheres

In [16]:
usgs_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

browser = init_browser()
browser.visit(usgs_url)

In [17]:
hemispheres = ["Cerberus","Schiaparelli","Syrtis","Valles"]
hemisphere_image_urls = []

# loop through each hemisphere and scrape the data
for hemi in hemispheres:
    new_dict = {}

    browser.click_link_by_partial_text(hemi)
    usgs_html = browser.html
    soup = BeautifulSoup(usgs_html, 'html.parser')
    new_dict["title"] = soup.find("h2").get_text().replace("Enhanced","").strip()
    new_dict["img_url"] = soup.find_all("div", class_="downloads")[0].find_all("a")[0]["href"]
    hemisphere_image_urls.append(new_dict)
    
    # go back to original page with all the hemispheres
    browser.back()

In [18]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [19]:
browser.quit()